In [ ]:
import json
import pandas as pd
import html
import re
from urllib import parse
import requests
from bs4 import BeautifulSoup as bs
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

REVIEW_PAGING_URL = "https://www.oliveyoung.co.kr/store/goods/getGdasListFormAjax.do"
SECTION_LIST = [{"NAME":"skin_toner", "dispCatNo":"1000001000100080001", "required_num": 350, "min_review_num_total": 3000, "min_review_num_12": 30},
                {"NAME":"lotion_emulsion", "dispCatNo":"1000001000100080002", "required_num": 200, "min_review_num_total": 3000, "min_review_num_12": 30},
                {"NAME":"all_in_one", "dispCatNo":"1000001000100080003", "required_num": 100, "min_review_num_total": 1000, "min_review_num_12": 10},
                {"NAME":"essense_serum", "dispCatNo":"1000001000100090001", "required_num": 300, "min_review_num_total": 3000, "min_review_num_12": 30},
                {"NAME":"cream", "dispCatNo":"1000001000100090002", "required_num": 400, "min_review_num_total": 3000, "min_review_num_12": 30},
                {"NAME":"face_oil", "dispCatNo":"1000001000100100002", "required_num": 50, "min_review_num_total": 1000, "min_review_num_12": 10}]
REIVEW_PAGE_HEADER = {"X-Requested-With": "XMLHttpRequest"}

def hex_to_char(hex_str):
    """ converts a single hex-encoded character 'FFFF' into the corresponding real character """
    return chr(int(hex_str, 16))

def unicode_decode(s):
    percent_u = re.compile(r"%u([0-9a-fA-F]{4})")
    decoded = percent_u.sub(lambda m: hex_to_char(m.group(1)), s)
    return decoded

Mounted at /content/drive


In [ ]:
# 해당 섹션에서 조건을 만족하는 모든 상품 리스트를 반환한다.
def extract_goods_list(dispCatNo, min_review_num_total, min_review_num_12):
    list_goodsNo = []
    num = 0
    review_num_12 = 0

    while True:
        num += 1
        url_item_list = f"https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo={dispCatNo}&fltDispCatNo=&prdSort=01&pageIdx={num}&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=2&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat1000001000100080001_Small&amplitudePageGubun=SMALL_CATE&t_page=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EA%B4%80&t_click=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EC%83%81%EC%84%B8_%EC%86%8C%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC&midCategory=%EB%A1%9C%EC%85%98%2F%EC%97%90%EB%A9%80%EC%A0%BC&smallCategory=%EC%86%8C_%EC%8A%A4%ED%82%A8%2F%ED%86%A0%EB%84%88&checkBrnds=&lastChkBrnd=&t_3rd_category_type=%EC%86%8C_%EC%8A%A4%ED%82%A8%2F%ED%86%A0%EB%84%88"
        page_item_list = requests.get(url_item_list)
        soup_item_list = bs(page_item_list.text, "html.parser")

        if (len(soup_item_list.select("#Contents > ul.cate_prd_list > li")) == 0):
            break

        for item in soup_item_list.select("#Contents > ul.cate_prd_list > li"):
            goodsNo = item.get("criteo-goods")[:-3]
            name = item.select(".tx_name")[0].text
            url_item_page_tmp = f"https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo={goodsNo}&dispCatNo=1000001000100080001&trackingCd=Cat1000001000100080001_Small&t_page=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EA%B4%80&t_click=%EC%8A%A4%ED%82%A8/%ED%86%A0%EB%84%88_%EC%86%8C_%EC%8A%A4%ED%82%A8/%ED%86%A0%EB%84%88__%EC%83%81%ED%92%88%EC%83%81%EC%84%B8&t_number=1"
            page_item_page_tmp = requests.get(url_item_page_tmp)
            soup_item_page_tmp = bs(page_item_page_tmp.text, "html.parser")

            review_num = int(soup_item_page_tmp.select(".goods_reputation > span")[0].text[1:-1].replace(",",""))

            # 1차 조건: 리뷰 개수가 min_review_num_total 개 이상일 것
            if (review_num >= min_review_num_total):

                review_num = [0,0,0]
                for point in range(1,3):
                    data = {"goodsNo": goodsNo, "gdasSort": "05", "itemNo": "all_search", "pageIdx": "1", "colData": "", "type": "", "point": point*2, "optionValue": "", "cTypeLength": "0", "keywordGdasSeqs": ""}
                    page_review_paging_tmp = requests.post(REVIEW_PAGING_URL, data=data)
                    soup_item_page_tmp = bs(page_review_paging_tmp.text, "html.parser")
                    review_page_num = len(soup_item_page_tmp.select("a")) + 1

                    if (review_page_num >= 11):
                        review_num[point] = 100
                    else:
                        url_review_page = f"https://www.oliveyoung.co.kr/store/goods/getGdasNewListJson.do?goodsNo={goodsNo}&gdasSort=05&itemNo=all_search&pageIdx={review_page_num}&colData=&keywordGdasSeqs=&type=&point={point*2}&hashTag=&optionValue=&cTypeLength=0"
                        page_review_page = requests.post(url_review_page, headers=REIVEW_PAGE_HEADER)
                        json_review_page = json.loads(page_review_page.text)
                        if (json_review_page.get("gdasList") == None or len(json_review_page.get("gdasList")) <= 0):
                            review_num[point] = 0
                        else:
                            review_num[point] = (review_page_num - 1) * 10 + len(json_review_page.get("gdasList"))

                # 2차 조건 : 1점짜리, 2점짜리 리뷰가 각 25개 이상이거나 두개 합쳐서 min_review_num_12 개 이상이면 사용한다
                # 가정 : 4,5점 짜리 리뷰는 항상 각각 25개가 넘는다
                # 전자를 만족하면 항상 후자를 만족하므로, 후자만 검사하면 된다
                if (sum(review_num) >= min_review_num_12):
                    list_goodsNo.append({"name": name, "goodsNo": goodsNo})
                    review_num_12 += min(sum(review_num), 50)

    return list_goodsNo

In [ ]:
def extract_reviews_from_goods_list(list_goodsNo, section, required_num):
    # pandas df로 데이터를 저장함 (같은 방식으로 load_csv로 불러와서 데이터 활용 가능)
    result_df = pd.DataFrame(columns=['section', 'goodsNo', 'name', 'point', 'label', 'review'])

    review_num_45 = 0
    review_num_12 = 0

    for item in list_goodsNo:
        # 각 상품 마다
        goodsNo = item["goodsNo"]

        # 별점 개수 (1~5) 별로 리뷰를 뽑음, 각 별점마다 최대 1000개씩 뽑히게 됨
        for point in (1,2,4,5):
            pageIdx = 0
            # 리뷰 페이지를 순회하며 각 페이지마다 10개씩 존재하는 리뷰를 뽑는다
            current_num = 0
            end = False
            while not end:
                pageIdx += 1
                url_review_page = f"https://www.oliveyoung.co.kr/store/goods/getGdasNewListJson.do?goodsNo={goodsNo}&gdasSort=05&itemNo=all_search&pageIdx={pageIdx}&colData=&keywordGdasSeqs=&type=&point={point*2}&hashTag=&optionValue=&cTypeLength=0"
                page_review_page = requests.post(url_review_page, headers=REIVEW_PAGE_HEADER)
                json_review_page = json.loads(page_review_page.text)
                if (json_review_page.get("gdasList") == None or len(json_review_page.get("gdasList")) <= 0):
                    break
                for review in json_review_page["gdasList"]:
                    if current_num >= 25 or \
                       ((point in (4,5)) and review_num_45 >= required_num) or \
                       ((point in (1,2)) and review_num_12 >= required_num):
                           end = True
                           break
                    elif (len(review["gdasCont"]) == 0) or ('img' in review["gdasCont"]):
                        continue

                    result_df.loc[len(result_df)] = [section, goodsNo, item["name"], int(review["gdasScrVal"])//2, int(review["gdasScrVal"]) // 6, re.sub("(<em>|</em>)", "", re.sub("(<br>|<br/>)", "\n", html.unescape(parse.unquote(unicode_decode(review["gdasCont"])))))]
                    current_num += 1
                    if point in (4,5):
                        review_num_45 += 1
                    elif point in (1,2):
                        review_num_12 += 1

    return result_df

In [ ]:
result_df_dict = {}

for sl in SECTION_LIST:
        list_goodsNo = extract_goods_list(sl["dispCatNo"], sl["min_review_num_total"], sl["min_review_num_12"])
        result_df_dict[sl["NAME"]] = extract_reviews_from_goods_list(list_goodsNo, sl["NAME"], sl["required_num"])

In [ ]:
# 최종적으로 결과 pandas DataFrame을 실행한 폴더 내에 csv 파일로 저장함
SAVE_PATH = "/content/drive/MyDrive/Colab Notebooks/"  # 원하는 경로로 수정 가능

for sl in SECTION_LIST:
    result_df_dict[sl["NAME"]].to_csv(f'{SAVE_PATH}/{sl["NAME"]}.csv',
                                      sep=',',
                                      na_rep='NaN',
                                      columns = ['section', 'goodsNo', 'name', 'point','label','review'], # columns to write
                                      index = False)

In [ ]:
test1, test2, test3, test4, test5, test6 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/skin_toner.csv"),pd.read_csv("/content/drive/MyDrive/Colab Notebooks/lotion_emulsion.csv"),pd.read_csv("/content/drive/MyDrive/Colab Notebooks/all_in_one.csv"),pd.read_csv("/content/drive/MyDrive/Colab Notebooks/essense_serum.csv"), pd.read_csv("/content/drive/MyDrive/Colab Notebooks/cream.csv"), pd.read_csv("/content/drive/MyDrive/Colab Notebooks/face_oil.csv")
print(len(test1), len(test2), len(test3), len(test4), len(test5), len(test6))

700 400 200 600 800 100


In [ ]:
# 모든 데이터프레임을 하나로 합칩니다.
merged_df = pd.concat([test1, test2, test3, test4, test5, test6], ignore_index=True)

# 합쳐진 데이터프레임을 Google Drive의 지정된 경로에 CSV 파일로 저장합니다.
merged_df.to_csv("/content/drive/MyDrive/Colab Notebooks/extracted_labeled_reviews.csv", index=False)